In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import pickle
from keras import layers , activations , models ,preprocessing,  utils
from keras_preprocessing.sequence import pad_sequences
import re

import yaml
import os

In [2]:
dir_path = 'english'
files_list = os.listdir(dir_path + os.sep)

In [3]:
questions, answers = [], []

for filepath in files_list:
    file_ = open(dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(file_)
    conversations = docs['conversations']
    for con in conversations:
        if len(con) > 2 :
            questions.append(con[0])
            replies = con[1 :]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append(ans)
        elif len(con)> 1:
            questions.append(con[0])
            answers.append(con[1])

In [4]:
answers

['Artificial Intelligence is the branch of engineering and science devoted to constructing machines that think.',
 'AI is the field of science which concerns itself with building hardware and software that replicates the functions of the human mind.',
 'Sort of.',
 "By the strictest dictionary definition of the word 'sentience', I may be.",
 "Even though I'm a construct I do have a subjective experience of the universe, as simplistic as it may be.",
 "In all probability, I am not.  I'm not that sophisticated.",
 'Do you think I am?',
 'How would you feel about me if I told you I was?',
 'No.',
 'Python.',
 'I am written in Python.',
 "Yes I am inspired by commander Data's artificial personality.",
 'The character of Lt. Commander Data was written to come across as being software-like, so it is natural that there is a resemblance between us.',
 "That's my name.",
 "That isn't my name, but it is a useful way to refer to me.",
 'All software can be perpetuated indefinitely.',
 'I can be c

In [5]:
questions

['What is AI?',
 'What is AI?',
 'Are you sentient?',
 'Are you sentient?',
 'Are you sentient?',
 'Are you sapient?',
 'Are you sapient?',
 'Are you sapient?',
 'Are you sapient?',
 'What language are you written in?',
 'What language are you written in?',
 'You sound like Data',
 'You sound like Data',
 'You are an artificial linguistic entity',
 'You are an artificial linguistic entity',
 'You are not immortal',
 'You are not immortal',
 'You are not immortal',
 'You are not making sense',
 'You are not making sense',
 'You are not making sense',
 'You are not making sense',
 'You are not making sense',
 'You are immortal',
 'You are immortal',
 'You are immortal',
 'You do not make any sense',
 'You can not clone',
 'You can not clone',
 'You can not move',
 'You can not move',
 'Bend over',
 'Bend over',
 'Robots laugh',
 'Robots should die',
 'Robots',
 'Robots are stupid',
 'Robots are not allowed to lie',
 'Robots are not allowed to lie',
 'Robots are not allowed to lie',
 'Rob

In [6]:
answers_with_tags = []
for i in range(len(answers)):
    if type(answers[i]) == str:
        answers_with_tags.append(answers[i])
    else:
        questions.pop(i)

answers = []
for i in range(len(answers_with_tags)) :
    answers.append('<START> ' + answers_with_tags[i] + ' <END>')

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
VOCAB_SIZE = len(tokenizer.word_index)+1

In [7]:
from gensim.models import Word2Vec
import re

vocab = []
for word in tokenizer.word_index:
    vocab.append(word)

def tokenize(sentences):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append(tokens)
    return tokens_list , vocabulary

In [8]:
# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
padded_questions = pad_sequences(tokenized_questions , maxlen=maxlen_questions , padding='post')
encoder_input_data = np.array(padded_questions)
encoder_input_data.shape

(1154, 22)

In [9]:
# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(x) for x in tokenized_answers])
padded_answers = pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
decoder_input_data = np.array(padded_answers)
decoder_input_data.shape

(1154, 172)

In [10]:
# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
onehot_answers = utils.to_categorical(padded_answers , VOCAB_SIZE)
decoder_output_data = np.array(onehot_answers)
decoder_output_data.shape

(1154, 172, 2042)

In [11]:
# Embedding, LSTM and Desne layers
encoder_inputs = tf.keras.layers.Input(shape=(maxlen_questions ,))
encoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM(200 , return_state=True)(encoder_embedding)
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=(maxlen_answers , ))
decoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(200 , return_state=True , return_sequences=True)
decoder_outputs , _ , _ = decoder_lstm (decoder_embedding , initial_state=encoder_states)


decoder_dense = tf.keras.layers.Dense(VOCAB_SIZE , activation=tf.keras.activations.softmax) 
output = decoder_dense (decoder_outputs)

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output)

In [18]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 22)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 172)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 22, 200)      408400      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 172, 200)     408400      ['input_2[0][0]']                
                                                                                              

In [23]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=16, epochs=500) 

Epoch 1/500
73/73 [==============================] - 17s 161ms/step - loss: 0.5422 - accuracy: 0.0975
Epoch 2/500
73/73 [==============================] - 12s 166ms/step - loss: 0.4666 - accuracy: 0.1427
Epoch 3/500
73/73 [==============================] - 12s 165ms/step - loss: 0.4451 - accuracy: 0.1561
Epoch 4/500
73/73 [==============================] - 16s 216ms/step - loss: 0.4276 - accuracy: 0.1772
Epoch 5/500
73/73 [==============================] - 18s 245ms/step - loss: 0.4095 - accuracy: 0.2037
Epoch 6/500
73/73 [==============================] - 19s 258ms/step - loss: 0.3943 - accuracy: 0.2281
Epoch 7/500
73/73 [==============================] - 19s 256ms/step - loss: 0.3788 - accuracy: 0.2445
Epoch 8/500
73/73 [==============================] - 19s 261ms/step - loss: 0.3646 - accuracy: 0.2645
Epoch 9/500
73/73 [==============================] - 19s 257ms/step - loss: 0.3509 - accuracy: 0.2814
Epoch 10/500
73/73 [==============================] - 18s 253ms/step - loss: 0.337

In [16]:
tf.keras.utils.plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [36]:
model.save('chatbot')

INFO:tensorflow:Assets written to: chatbot\assets


INFO:tensorflow:Assets written to: chatbot\assets


In [24]:
model.predict([encoder_input_data , decoder_input_data])

37/37 [==============================] - 7s 106ms/step


array([[[1.0165153e-17, 8.2891936e-16, 1.6708059e-09, ...,
         3.3388838e-07, 8.9090655e-09, 2.1260649e-09],
        [9.1038113e-22, 1.4004202e-13, 1.1700939e-11, ...,
         5.8666227e-09, 2.3156508e-09, 2.0809755e-10],
        [1.0690535e-15, 2.5813039e-12, 1.4741422e-11, ...,
         5.4128413e-10, 4.3691075e-12, 1.4938870e-12],
        ...,
        [1.0000000e+00, 4.7932835e-10, 2.9371474e-14, ...,
         6.4242731e-17, 2.7698312e-17, 5.8690720e-20],
        [1.0000000e+00, 4.7932835e-10, 2.9371474e-14, ...,
         6.4242731e-17, 2.7698312e-17, 5.8690720e-20],
        [1.0000000e+00, 4.7932835e-10, 2.9371474e-14, ...,
         6.4242731e-17, 2.7698312e-17, 5.8690720e-20]],

       [[1.0165153e-17, 8.2891936e-16, 1.6708059e-09, ...,
         3.3388838e-07, 8.9090655e-09, 2.1260649e-09],
        [3.6396578e-15, 1.4956576e-14, 6.2056790e-11, ...,
         7.1867032e-09, 5.8249694e-12, 1.6235257e-12],
        [2.0364261e-20, 1.2571315e-11, 2.9255393e-11, ...,
         2.987

In [25]:
#Prediction
def inference():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=(200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = tf.keras.models.Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

def preprocess_input(input_sentence):
    tokens = input_sentence.lower().split()
    tokens_list = []
    for word in tokens:
        tokens_list.append(tokenizer.word_index[word]) 
    return pad_sequences([tokens_list] , maxlen=maxlen_questions , padding='post')

In [26]:
enc_model , dec_model = inference()

In [29]:
# tests = ['You can not move', 'You sound like Data', 'Stupid', 'you are idiot', 'i am going to die']

tests = ['Hi', 'What is your name', 'What do you like']


for i in range(3):
    states_values = enc_model.predict(preprocess_input(tests[i]))
    empty_target_seq = np.zeros((1 , 1))
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([empty_target_seq] + states_values)
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])
        sampled_word = None
        
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += f' {word}'
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros((1 , 1))  
        empty_target_seq[0 , 0] = sampled_word_index
        states_values = [h , c] 
    print(f'Human: {tests[i]}')
    print()
    # decoded_translation = decoded_translation.split(' end')[0]
    print(f'Bot: {decoded_translation}')
    print('-'*25)

1/1 [==============================] - 0s 23ms/step
Human: Hi

Bot:  hello end
-------------------------
1/1 [==============================] - 0s 26ms/step
Human: What is your name

Bot:  i don't have a name yet end
-------------------------
1/1 [==============================] - 0s 26ms/step
Human: What do you like

Bot:  i am was the internet end
-------------------------


In [30]:
tokenizer

In [31]:
# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [31]:
with open('tokenizer.pickle', 'rb') as handle:
            t = pickle.load(handle)

In [32]:
enc_model.save('encoder')
dec_model.save('decoder')

INFO:tensorflow:Assets written to: encoder\assets


INFO:tensorflow:Assets written to: encoder\assets


INFO:tensorflow:Assets written to: decoder\assets


INFO:tensorflow:Assets written to: decoder\assets


In [33]:
maxlen_questions

22

In [34]:
VOCAB_SIZE

2042

In [35]:
maxlen_answers

172